Файл повторяет парсинг позиций в точности до $\pm$10 строк.

In [1]:
import pandas as pd
import requests as rq
import numpy as np
from datetime import datetime, timedelta
from concurrent.futures import ThreadPoolExecutor
import os

# Демонстрация того, как будет происходить процесс совмещения на двух днях

In [2]:
# Ссылка на данные
url1 = "https://iss.moex.com/iss/history/engines/stock/markets/shares/sessions/2/securities/LKOH.json?from=2023-11-07&till=2023-11-07&table_type=full"
url2 = "https://iss.moex.com/iss/history/engines/stock/markets/shares/sessions/2/securities/LKOH.json?from=2023-11-08&till=2023-11-08&table_type=full"

In [3]:
# Отправляем GET-запрос по указанной ссылке
response = rq.get(url1)

# Проверяем статус ответа
if response.status_code == 200:
    # Загружаем данные из ответа в переменную data в случае успеха
    data = response.json()
else:
    print("Ошибка при получении данных")
    

columns = response.json()["history"]['columns']

data = response.json()["history"]["data"]

df1 = pd.DataFrame(data, columns=columns)


# Отправляем GET-запрос по указанной ссылке
response = rq.get(url2)

# Проверяем статус ответа
if response.status_code == 200:
    # Загружаем данные из ответа в переменную data в случае успеха
    data = response.json()
else:
    print("Ошибка при получении данных")


columns = response.json()["history"]['columns']

data = response.json()["history"]["data"]

df2 = pd.DataFrame(data, columns=columns)

In [4]:
print(df1.sort_values(by=['TRADEDATE']).reset_index().drop('index', axis=1))

    TRADEDATE SECID                   BOARDNAME BOARDID SHORTNAME  \
0  2023-11-07  LKOH  Т+: Акции и ДР - безадрес.    TQBR    ЛУКОЙЛ   

      REGNUMBER          ISIN LISTNAME  FACEVALUE CURRENCYID  ...  \
0  1-01-00077-A  RU0009024277        1      0.025        SUR  ...   

  MARKETPRICE3  MARKETPRICE3TRADESVALUE DECIMALS          TYPE  \
0         None                        0        1  common_share   

   CLOSEAUCTIONPRICE  WAVAL  LASTPRICE  MARKETPRICE3TRADESVALUECUR  \
0                  0      0     7333.5                        None   

  MARKETPRICE3CUR  TRADINGSESSION  
0            None               2  

[1 rows x 53 columns]


In [5]:
frames = [df1, df2]

result = pd.concat(frames).sort_values(by=['TRADEDATE']).reset_index().drop('index', axis=1)
filename = "aboba_date.csv"
result.to_csv(filename, sep=',', index=False, encoding='utf-8')

print(result)

    TRADEDATE SECID                   BOARDNAME BOARDID SHORTNAME  \
0  2023-11-07  LKOH  Т+: Акции и ДР - безадрес.    TQBR    ЛУКОЙЛ   
1  2023-11-08  LKOH  Т+: Акции и ДР - безадрес.    TQBR    ЛУКОЙЛ   

      REGNUMBER          ISIN LISTNAME  FACEVALUE CURRENCYID  ...  \
0  1-01-00077-A  RU0009024277        1      0.025        SUR  ...   
1  1-01-00077-A  RU0009024277        1      0.025        SUR  ...   

  MARKETPRICE3  MARKETPRICE3TRADESVALUE DECIMALS          TYPE  \
0         None                        0        1  common_share   
1         None                        0        1  common_share   

   CLOSEAUCTIONPRICE  WAVAL  LASTPRICE  MARKETPRICE3TRADESVALUECUR  \
0                  0      0     7333.5                        None   
1                  0      0     7295.0                        None   

  MARKETPRICE3CUR  TRADINGSESSION  
0            None               2  
1            None               2  

[2 rows x 53 columns]


Далее будет пробегать по массиву дат и таким образом сформируем всю таблицу

Ниже я задаю массив всех дат

# Парсинг данных данных

Сначала мы создаём массив, который содержить все даты, за которые мы хотим получить данные

In [6]:
start_date = datetime(2020, 5, 8)
end_date = datetime.now()

current_date = start_date
array_date = []
while current_date <= end_date:
    array_date += [current_date.strftime("%Y-%m-%d")]
    current_date += timedelta(days=1)

Тут мы формируем массив из дней, а затем, если сервер работает хорошо, то возвращаем данные за весь промежуток от 5 августа 2020 до сегодняшнего дня. 

Если сервер перестаёт отвечать, тогда полученные данные мы собираем в файл и при повторном запуске программа проверить наши файлы и начнёт с последнего записанного дня.

В качестве отладки печатается дата дня, который мы сейчас получаем. 

In [7]:
def add_new_day(date_today: str, ticker_name: str): 
    url = f"https://iss.moex.com/iss/history/engines/stock/markets/shares/sessions/2/securities/{ticker_name}.json?from={date_today}&till={date_today}&table_type=full" 
 
    while True: 
        try: 
            # Ваш код, который может вызвать ошибку 
            response = rq.get(url) 
            # break  # Если ошибки не возникло, выходим из цикла 
        except Exception as e: 
            # Обработка других исключений 
            print(f"Произошла ошибка: {e}") 
        else: 
            break 
 
    # Проверяем статус ответа 
    if response.status_code != 200: 
        print("Ошибка при получении данных") 
        add_new_day(date_today) 
 
    df_for_add = pd.DataFrame(response.json()["history"]["data"], columns=response.json()["history"]['columns']) 
    df_for_add.sort_values(by=['TRADEDATE']).reset_index().drop('index', axis=1) 
    
 
    return df_for_add 
 
def process_ticker(ticker_name): 
    if (os.path.exists(f"{ticker_name}_full_date_price.csv") or os.path.exists(f"{ticker_dict[ticker_name]}_full_date_price.csv")):
        print(f"Файл '{ticker_name}_full_date_price.csv' существует.") 
        df = pd.read_csv(f"{ticker_name}_full_date_price.csv") 
        last_valid_date_index = df['TRADEDATE'].last_valid_index() 
        last_valid_date_value  = df['TRADEDATE'][last_valid_date_index] 
        date_tomorrow = (datetime.strptime(last_valid_date_value, '%Y-%m-%d') + timedelta(days=1)).strftime("%Y-%m-%d") 
        print(date_tomorrow) 
    # Вывод первых нескольких строк DataFrame для проверки 
        if(not date_tomorrow in array_date): 
            pass 
        else: 
            df_array = [df] 
            for date in array_date[array_date.index(date_tomorrow)+1:]: 
                df_array += [add_new_day(date, ticker_name)] 
                print(date) 
            df = pd.concat(df_array).sort_values(by=['TRADEDATE']).reset_index().drop('index', axis=1) 
            print(df) 
            filename = f"{ticker_name}_full_date_price.csv" 
            df.to_csv(filename, sep=',', index=False, encoding='utf-8') 
    else: 
        print(f"Файл '{ticker_name}_full_date_price.csv' не существует.") 
        df_array = [] 
        for date in array_date: 
            df_array += [add_new_day(date, ticker_name)] 
            print(date) 
        df = pd.concat(df_array).sort_values(by=['TRADEDATE']).reset_index().drop('index', axis=1) 
        print(df) 
        filename = f"{ticker_name}_full_date_price.csv" 
        df.to_csv(filename, sep=',', index=False, encoding='utf-8') 
        
ticker_names = ['SBER', 'GAZP', 'LKOH', 'VTBR', 'ROSN', 'MGNT', 'AFLT', 'ALRS', 'SNGS', 'YNDX', 'TATN', 'NLMK', 'HYDR', 'MOEX', 'FIVE', 'GMKN', 'MAGN']
short_ticker_names = ['sr', 'gz', 'lk', 'vb', 'rn', 'mn', 'af', 'al', 'sn', 'yn', 'tt', 'nm', 'hy', 'me', 'fv', 'gk', 'mg', 'ml']

ticker_dict = dict(zip(ticker_names, short_ticker_names))

with ThreadPoolExecutor() as executor: 
    executor.map(process_ticker, ticker_names)

Файл 'SBER_full_date_price.csv' существует.
Файл 'GAZP_full_date_price.csv' существует.
Файл 'LKOH_full_date_price.csv' существует.
Файл 'ROSN_full_date_price.csv' существует.
Файл 'MGNT_full_date_price.csv' существует.
Файл 'VTBR_full_date_price.csv' существует.
Файл 'SNGS_full_date_price.csv' существует.
Файл 'YNDX_full_date_price.csv' существует.
Файл 'AFLT_full_date_price.csv' существует.
Файл 'ALRS_full_date_price.csv' существует.
Файл 'TATN_full_date_price.csv' существует.
Файл 'HYDR_full_date_price.csv' существует.
Файл 'NLMK_full_date_price.csv' существует.
Файл 'MOEX_full_date_price.csv' существует.
Файл 'GMKN_full_date_price.csv' существует.
Файл 'FIVE_full_date_price.csv' существует.
Файл 'MAGN_full_date_price.csv' существует.


Удаление повторяющихся данных (в силу того, как МосБиржа возвращала данные (в выходные дни вместо пустого запроса она возвращает последнюю цену за последний рабочий день, то есть 14*20 одинаковых записей, в силу чего эта ячейка позволяла сэкономить, примерно, 300 МБ))

In [8]:
import os 
 
def rename_files(ticker_names): 
    # Путь к папке с файлами 
    folder_path = '.' 
 
    # Получаем список файлов в папке 
    files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))] 
 
    for file_name in files: 
        # Проверяем, что файл имеет нужное расширение и соответствует шаблону 
        if file_name.endswith('.csv') and any(ticker in file_name for ticker in ticker_names): 
            # Извлекаем имя тикера из имени файла 
            for ticker, short_ticker in zip(ticker_names, ['sr', 'gz', 'lk', 'vb', 'rn', 'mn', 'af', 'al', 'sn', 'yn', 'tt', 'nm', 'hy', 'me', 'fv', 'gk', 'mg', 'ml']): 
                if ticker in file_name: 
                    # Формируем новое имя файла 
                    new_file_name = file_name.replace(f"{ticker}_full_date_price", f"{short_ticker}_full_date_price") 
                    # Составляем полные пути к старому и новому файлу 
                    old_file_path = os.path.join(folder_path, file_name) 
                    new_file_path = os.path.join(folder_path, new_file_name) 
                    # Переименовываем файл 
                    os.rename(old_file_path, new_file_path) 
                    print(f'Файл {file_name} переименован в {new_file_name}') 
                    break 
 
rename_files(['SBER', 'GAZP', 'LKOH', 'VTBR', 'ROSN', 'MGNT', 'AFLT', 'ALRS', 'SNGS', 'YNDX', 'TATN', 'NLMK', 'HYDR', 'MOEX', 'FIVE', 'GMKN', 'MAGN'])

In [9]:
ticker_names=['sr', 'gz', 'lk', 'vb', 'rn', 'mn', 'af', 'al', 'sn', 'yn', 'tt', 'nm', 'hy', 'me', 'fv', 'gk', 'mg']

for ticker_name in ticker_names:
    your_file = f"{ticker_name}_full_date_price.csv" 
# Загрузка данных
    df = pd.read_csv(your_file)

# Удаление повторяющихся строк
    unique_rows = df.drop_duplicates()

# Сохранение результата
    unique_rows.to_csv(your_file, index=False)

Итоговые данные не идеальны, поэтому мы их будем обрабатывать в других данных.